In [6]:
import pandas as pd

import src.common.methods_functions as methods_functions

selected_treasury_tables = ['v2/accounting/od/debt_to_penny']
selected_fred_tables = ['SP500', 'NASDAQCOM']
start_year = 1980
end_year = 2023

treasury_columns = {
    'v1/accounting/dts/dts_table_1': ['record_date', 'close_today_bal', 'account_type'],
    'v2/accounting/od/debt_to_penny': ['record_date', 'tot_pub_debt_out_amt']
}

fred_column_names = {
    'SP500': {'value': "S&P 500 Value ($)"},
    'NASDAQCOM': {'value': "Nasdaq Composite Value ($)"}
}

treasury_column_names = {
    'v1/accounting/dts/dts_table_1': {'record_date': 'date', 'close_today_bal': 'Daily Treasury Balance (Millions $)'},
    'v2/accounting/od/debt_to_penny': {'record_date': 'date', 'tot_pub_debt_out_amt': 'Outstanding US Debt ($)'}
}

all_years_string = ','.join(str(year) for year in range(start_year, end_year + 1))

fred_api = methods_functions.DataFetcher.fetch_fred_data(selected_fred_tables, start_year, end_year, 'lin', 'd', 'lin')
treasury_api = methods_functions.DataFetcher.fetch_treasury_data(selected_treasury_tables, all_years_string)

all_dfs = []
for table in selected_fred_tables:
    fred_df = methods_functions.process_fred_table(fred_api, table, fred_column_names)
    if fred_df is not None: all_dfs.append(fred_df)
for table in selected_treasury_tables:
    treasury_df = methods_functions.process_treasury_table(treasury_api, table, treasury_columns, treasury_column_names)
    if treasury_df is not None: all_dfs.append(treasury_df)

table_df = all_dfs[0]
for i in range(1, len(all_dfs)):
    table_df = pd.merge(table_df, all_dfs[i], on='date')

print(table_df)


            date S&P 500 Value ($) Nasdaq Composite Value ($)  \
0     2013-07-15           1682.50                    3607.49   
1     2013-07-16           1676.26                    3598.50   
2     2013-07-17           1680.91                    3610.00   
3     2013-07-18           1689.37                    3611.28   
4     2013-07-19           1692.09                    3587.61   
...          ...               ...                        ...   
2373  2022-12-23           3844.82                   10497.86   
2374  2022-12-27           3829.25                   10353.23   
2375  2022-12-28           3783.22                   10213.29   
2376  2022-12-29           3849.28                   10478.09   
2377  2022-12-30            3839.5                   10466.48   

     Outstanding US Debt ($)  
0          16738188365630.03  
1          16738177765933.41  
2          16738167165761.57  
3          16738168990385.79  
4          16738158460368.61  
...                      ...  
23

In [8]:
print(table_df.isnull().values.any())
print(table_df.isnull().sum())

False
date                          0
S&P 500 Value ($)             0
Nasdaq Composite Value ($)    0
Outstanding US Debt ($)       0
dtype: int64


In [2]:
import pandas as pd
import numpy as np

table_df = pd.Series([2,3,np.nan,7,"The Hobbit"])

table_df.isnull().sum()

1

In [ ]:
import psycopg2

import my_config

conn = psycopg2.connect(my_config.DB_CREDENTIALS)

query = '''
    SELECT
        table_name
    FROM
        information_schema.tables
    WHERE
        table_schema = 'public';
'''

cursor = conn.cursor()
cursor.execute(query)

table_names = cursor.fetchall()
cursor.close()
conn.close()

for table in table_names:
    print(table[0])

In [ ]:
import json

import pandas as pd
import requests

import my_config
from src.common.methods_functions import RestAPI

class RestAPI:
    def __init__(self, base_url, endpoints):
        self.base_url = base_url
        self.endpoints = endpoints
        self.data = {}

    def fetch_data(self):
        for key, endpoint in self.endpoints.items():
            url = self.base_url + endpoint
            response = requests.get(url)
            if response.status_code == 200: 
                self.data[key] = response.json()
            else: 
                print(f"Error: {response.status_code}")
    
    def save_to_json(data, file_name: str):
        with open(file_name + ".json", "w") as file:
            json.dump(data, file, indent=4)

def process_fred_table(api, table, column_names, monthly=False, quarterly=False):
    try:
        data = api.data[table]['observations']
        df = pd.DataFrame(data, columns=['date', 'value']).rename(columns=column_names[table])
        if monthly:
            df['date'] = pd.to_datetime(df['date']).dt.to_period('M')
        if quarterly:
            df['date'] = pd.to_datetime(df['date'])
            df['date'] = df['date'].dt.year.astype(str) + 'Q' + df['date'].dt.quarter.astype(str)
        return df
    except KeyError:
        print("f")

def process_bea_table(api, table, filter_metrics, table_names, monthly=False):
    try:
        data = api.data[table]['BEAAPI']['Results']['Data']
        df = pd.DataFrame(data, columns=['TimePeriod', 'DataValue', 'METRIC_NAME', 'LineDescription'])
        df = df.loc[df['LineDescription'] == filter_metrics[table]].drop(columns=['LineDescription'])
    except KeyError:
        print("rip")

    # Pivoting the table to have seperate columns for each distinct calculation method of the general economic metric.
    sliced_dfs = [
        df.loc[df['METRIC_NAME'] == metric]
        .drop(columns=['METRIC_NAME'])
        .reset_index(drop=True)
        .rename(columns={'DataValue': f'{table_names[table]} - ' + metric})
        for metric in df['METRIC_NAME'].unique()
    ]
    pivoted_df = sliced_dfs[0]
    for i in range(1, len(sliced_dfs)):
        pivoted_df = pd.merge(pivoted_df, sliced_dfs[i], on='TimePeriod')
    pivoted_df.rename(columns={'TimePeriod': 'date'}, inplace=True)

    if monthly:
            pivoted_df['date'] = pd.to_datetime(pivoted_df['date'], format='%YM%m').dt.to_period('M')
    return pivoted_df

Requesting BEA table names

In [ ]:
bea_base_url = f"https://apps.bea.gov/api/data/"

bea_all_endpoints = {
    'all_nipa': f"?&UserID={my_config.BEA_KEY}&DataSetName=NIPA&method=GetParameterValues&ParameterName=TableName",
    'all_fixedassets': f"?&UserID={my_config.BEA_KEY}&method=GetParameterValues&DataSetName=FixedAssets&ParameterName=TableName",
    'all_gdpindustry': f"?&UserID={my_config.BEA_KEY}&method=GetParameterValues&DataSetName=GDPbyIndustry&ParameterName=TableID",
    'all_io': f"?&UserID={my_config.BEA_KEY}&method=GetParameterValues&DataSetName=InputOutput&ParameterName=TableID"
}

bea_api = RestAPI(bea_base_url, bea_all_endpoints)
bea_api.fetch_data()

print(json.dumps(bea_api.data['all_nipa']['BEAAPI']['Results'], indent=4))

Checking table contents for each API

In [ ]:
# BEA
start_year = 2019
end_year = 2020
bea_frequency = 'M'
bea_dataset = 'NIPA'
selected_bea_tables = ['T20804']
bea_years = ','.join(str(year) for year in range(start_year, end_year + 1))

bea_base_url = f"https://apps.bea.gov/api/data/"

bea_endpoints = {
    f'{table}': (f"?&UserID={my_config.BEA_KEY}"
                 f"&method=GetData"
                 f"&DataSetName={bea_dataset}"
                 f"&TableName={table}"
                 f"&Frequency={bea_frequency}"
                 f"&Year={bea_years}")
    for table in selected_bea_tables
}

bea_api = RestAPI(bea_base_url, bea_endpoints)
bea_api.fetch_data()

print(json.dumps(bea_api.data['T20804']['BEAAPI']['Results']['Data'], indent=4))

In [ ]:
treasury_tables = {
    'interest_rates': 'v2/accounting/od/avg_interest_rates',
    'cash_balance': 'v1/accounting/dts/dts_table_1',
    'transactions': 'v1/accounting/dts/dts_table_2',
    'us_debt': 'v2/accounting/od/debt_to_penny',
    'balance_sheet': 'v2/accounting/od/balance_sheets'
}

In [ ]:
# US Treasury
start_year = 2022
end_year = 2023
selected_treasury_tables = ['v2/accounting/od/avg_interest_rates']
all_years_string = ','.join(str(year) for year in range(start_year, end_year + 1))

treasury_base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/"

treasury_endpoints = {
    f'{table}': f"{table}?&filter=record_fiscal_year:in:({all_years_string})&page[size]=10000" for table in selected_treasury_tables
}

treasury_api = RestAPI(treasury_base_url, treasury_endpoints)
treasury_api.fetch_data()

print(json.dumps(treasury_api.data['v2/accounting/od/avg_interest_rates']['data'], indent=4))

In [ ]:
# FRED
fred_units = "lin"
fred_frequency = "m"
selected_fred_tables = ["UNRATE"]
fred_start_date = "2005-01-01"
fred_end_date = "2020-01-01"

fred_base_url = "https://api.stlouisfed.org/fred/series/observations"

fred_endpoints = {
    f'{table}': (f"?series_id={table}"
                      f"&api_key={my_config.FRED_KEY}"
                      f"&file_type=json"
                      f"&observation_start={fred_start_date}"
                      f"&observation_end={fred_end_date}"
                      f"&units={fred_units}"
                      f"&frequency={fred_frequency}"
                      f"&aggregation_method=avg")
    for table in selected_fred_tables
}
 
fred_api = RestAPI(fred_base_url, fred_endpoints)
fred_api.fetch_data()

print(json.dumps(fred_api.data['UNRATE'], indent=4))

In [ ]:
# US Bureau of Labor Statistics - really slow
labor_dates = {
    'startyear': '2005',
    'endyear': '2020'
}

labor_base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

labor_endpoints = {
    'unemployment_rate': 'LNS14000000',
    # 'cpi': 'CUUR0000SA0'
}

labor_api = RestAPI(labor_base_url, labor_endpoints)
labor_api.fetch_data()

print(json.dumps(labor_api.data['unemployment_rate'], indent=4))

DataFrame creation and Testing

In [ ]:
filter_metric = {
    'T10101': "Gross domestic product",
    'T10105': "Gross domestic product",
    'T10107': "Gross domestic product",
    'T20100': "Personal income",
    'T20307': "Personal consumption expenditures (PCE)",
    'T20301': "Personal consumption expenditures (PCE)",
    'T20304': "Personal consumption expenditures (PCE)",
    'T20600': "Personal income"
}

bea_column_names = {
    'T10101': "Real GDP (Quarterly Change)",
    'T10105': "Total GDP (Millions $)",
    'T10107': "GDP (Quarterly Change)",
    'T20100': "Personal Income (Millions $)",
    'T20307': "PCE (Quarterly Change)",
    'T20301': "Real PCE (Quarterly Change)",
    'T20304': "PCEPI",
    'T20600': 'Personal Income (Millions $)'
}

fred_column_names = {
    'CPIAUCSL': {'value': "CPI"},
    'PAYEMS': {'value': 'Nonfarm Payrolls (Thousands of Persons)'},
    'UNRATE': {'value': 'Unemployment Rate'},
    'USSTHPI': {'value': "House Price Index"}
}

all_dfs = []
for table in selected_fred_tables:
    fred_df = process_fred_table(fred_api, table, fred_column_names, monthly=True)
    if fred_df is not None: all_dfs.append(fred_df)
for table in selected_bea_tables:
    bea_df = process_bea_table(bea_api, table, filter_metric, bea_column_names, monthly=True)
    if bea_df is not None: all_dfs.append(bea_df)

table_df = all_dfs[0]
for i in range(1, len(all_dfs)):
    table_df = pd.merge(table_df, all_dfs[i], on='date')

print(table_df)

Catalogued tables

In [ ]:
# BEA Tables
all_bea_tables = {
    "T10503": "Real Gross Domestic Product, Expanded Detail, Quantity Indexes (A) (Q)",
    "T10105": "Gross Domestic Product (A) (Q)",
    "T10505": "Gross Domestic Product, Expanded Detail (A) (Q)",
    "T80103": "Real Gross Domestic Product, Quantity Indexes, Not Seasonally Adjusted (Q)",
    "T80104": "Price Indexes for Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80105": "Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80106": "Real Gross Domestic Product, Chained Dollars, Not Seasonally Adjusted (Q)",
    "T80200": "Gross Domestic Income by Type of Income, Not Seasonally Adjusted (Q)",
    "T11000": "Gross Domestic Income by Type of Income (A) (Q)",
    "T11000": "Gross Domestic Income by Type of Income (A) (Q)",

    "T20301": "Percent Change From Preceding Period in Real Personal Consumption Expenditures by Major Type of Product (A) (Q)",
    "T20303": "Real Personal Consumption Expenditures by Major Type of Product, Quantity Indexes (A) (Q)",
    "T20304": "Price Indexes for Personal Consumption Expenditures by Major Type of Product (A) (Q)",
    "T20305": "Personal Consumption Expenditures by Major Type of Product (A) (Q)",
    "T20306": "Real Personal Consumption Expenditures by Major Type of Product, Chained Dollars (A) (Q)",
    "T20307": "Percent Change From Preceding Period in Prices for Personal Consumption Expenditures by Major Type of Product (A) (Q)",

    "T20100": "Personal Income and Its Disposition (A) (Q)",    
    "T20600": "Personal Income and Its Disposition, Monthly (M)",
    "T20200A": "Wages and Salaries by Industry (A) (Q)",
    "T20200B": "Wages and Salaries by Industry (A) (Q)",
    "T20700A": "Wages and Salaries by Industry, Monthly (M)",
    "T20700B": "Wages and Salaries by Industry, Monthly (M)",

    "T40201": "Percent Change From Preceding Period in Real Exports and in Real Imports of Goods and Services by Type of Product (A) (Q)",
    "T40203": "Real Exports and Imports of Goods and Services by Type of Product, Quantity Indexes (A) (Q)",
    "T40204": "Price Indexes for Exports and Imports of Goods and Services by Type of Product (A) (Q)",
    "T40205": "Exports and Imports of Goods and Services by Type of Product (A) (Q)",

    "T50100": "Saving and Investment by Sector (A) (Q)",
    "T50203": "Real Gross and Net Domestic Investment by Major Type, Quantity Indexes (A)",
    "T50205": "Gross and Net Domestic Investment by Major Type (A)",
    "T50301": "Percent Change From Preceding Period in Real Private Fixed Investment by Type (A) (Q)",

    "T50705A": "Change in Private Inventories by Industry (A) (Q)",
    "T50705B": "Change in Private Inventories by Industry (A) (Q)",

    "T61600A": "Corporate Profits by Industry (A)",
    "T61600B": "Corporate Profits by Industry (A) (Q)",
    "T61600C": "Corporate Profits by Industry (A) (Q)",
    "T61600D": "Corporate Profits by Industry (A) (Q)",

    "T70201A": "Percent Change From Preceding Period in Real Auto Output (A) (Q)",
    "T70201B": "Percent Change From Preceding Period in Real Motor Vehicle Output (A) (Q)",
    "T70203A": "Real Auto Output, Quantity Indexes (A) (Q)",
    "T70203B": "Real Motor Vehicle Output, Quantity Indexes (A) (Q)",
    "T70204A": "Price Indexes for Auto Output (A) (Q)",
    "T70204B": "Price Indexes for Motor Vehicle Output (A) (Q)",
    "T70205A": "Auto Output (A) (Q)",
    "T70205B": "Motor Vehicle Output (A) (Q)",

    "T80300": "Federal Government Current Receipts and Expenditures, Not Seasonally Adjusted (Q)"
}

gdp_percent_changes = {
    "T10101" : "Percent Change From Preceding Period in Real Gross Domestic Product (A) (Q)",
    "T10107": "Percent Change From Preceding Period in Prices for Gross Domestic Product (A) (Q)",
    "T10108": "Contributions to Percent Change in the Gross Domestic Product Price Index (A) (Q)",
    "T10501": "Percent Change From Preceding Period in Real Gross Domestic Product, Expanded Detail (A) (Q)",
    "T80111": "Real Gross Domestic Product: Percent Change From Quarter One Year Ago, Not Seasonally Adjusted (Q)",
    "T80103": "Real Gross Domestic Product, Quantity Indexes, Not Seasonally Adjusted (Q)",
    "T80104": "Price Indexes for Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80105": "Gross Domestic Product, Not Seasonally Adjusted (Q)",
    "T80106": "Real Gross Domestic Product, Chained Dollars, Not Seasonally Adjusted (Q)",
}

bea_monthly_tables = {
    "T20805": "Personal Consumption Expenditures by Major Type of Product, Monthly (M)",
    "T20804": "Price Indexes for Personal Consumption Expenditures by Major Type of Product, Monthly (M)",
    "T20801": "Percent Change From Preceding Period in Real Personal Consumption Expenditures by Major Type of Product, Monthly (M)",
    "T20807": "Percent Change From Preceding Period in Prices for Personal Consumption Expenditures by Major Type of Product, Monthly (M)",
    "T20803": "Real Personal Consumption Expenditures by Major Type of Product, Monthly, Quantity Indexes (M)",
}

In [ ]:
# US Treasury Tables
treasury_tables = {
    'interest_rates': 'v2/accounting/od/avg_interest_rates',
    'cash_balance': 'v1/accounting/dts/dts_table_1',
    'transactions': 'v1/accounting/dts/dts_table_2',
    'us_debt': 'v2/accounting/od/debt_to_penny',
    'balance_sheet': 'v2/accounting/od/balance_sheets'
}

Misc.

In [ ]:
def add_date_table(table, dataset_list):
    date_table_added = table not in dataset_list
    if date_table_added: dataset_list += [table]
    return date_table_added

def create_date_series(date_name, data):
    periods = [item[date_name] for item in data]
    unique_periods = list(set(periods))
    date_sr = pd.Series(unique_periods, name='date').sort_values().reset_index(drop=True)
    return date_sr